需要养成写注释的好习惯, 所以强制要求自己在每个Cell之前必须要有一个Markdown单元格简述内容

对, 同时要打开"中文模式英文标点"防止高血压操作

接下来的cell是简单的依赖库的导入

注意, 刚刚了解到ee.Initialize()的令牌是只有一周的有效期的, 这也意味着需要一周进行一次Authenticate

In [9]:
import ee
from ee_jupyter.ipyleaflet import Map
import ipyleaflet as ipyl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("Authenticating using Notebook auth...")
if os.path.exists(ee.oauth.get_credentials_path()) is False:
    ee.Authenticate()
else:
    print('\N{check mark} '
            'Previously created authentication credentials were found.')
    ee.Initialize()

接下来是导入Landsat8的影像,在GEE上创建了一个polygon, 

将这个polygon的JSON复制下来, 创建了aoi对象

In [28]:
ls8=ee.ImageCollection("LANDSAT/LC08/C02/T1_RT")
aoi=ee.Geometry({
  "type": "Polygon",
  "coordinates": [
    [
      [
        89.92270815646162,
        26.012618981109068
      ],
      [
        118.92661440646168,
        26.012618981109068
      ],
      [
        118.92661440646168,
        42.468256766486284
      ],
      [
        89.92270815646162,
        42.468256766486284
      ],
      [
        89.92270815646162,
        26.012618981109068
      ]
    ]
  ]
})

aoi.getInfo()

{'type': 'Polygon',
 'coordinates': [[[89.92270815646162, 26.012618981109068],
   [118.92661440646168, 26.012618981109068],
   [118.92661440646168, 42.468256766486284],
   [89.92270815646162, 42.468256766486284],
   [89.92270815646162, 26.012618981109068]]]}

通过读取Band信息可以知道:

Band: QA_PIXEL:

- Bit 0: Fill
    - 0: Image data
    - 1: Fill data
- Bit 1: Dilated Cloud
    - 0: Cloud is not dilated or no cloud
  - 1: cloud dilation
- Bit 2: Unused
- Bit 3: Cloud
  - 0: Cloud confidence is not high
  - 1: High confidence cloud
- Bit 4: Cloud Shadow
  - 0: Cloud Shadow Confidence is not high
  - 1: High confidence cloud shadow
- Bit 5: Snow
  - 0: Snow/Ice Confidence is not high
  - 1: High confidence snow cover
- Bit 6: Clear
  - 0: Cloud or Dilated Cloud bits are set
  - 1: Cloud and Dilated Cloud bits are not set
- Bit 7: Water
  - 0: Land or cloud
  - 1: Water
- Bits 8-9: Cloud Confidence
  - 0: No confidence level set
  - 1: Low confidence
  - 2: Medium confidence
  - 3: High confidence
- Bits 10-11: Cloud Shadow Confidence
  - 0: No confidence level set
  - 1: Low confidence
  - 2: Reserved
  - 3: High confidence
- Bits 12-13: Snow / Ice Confidence
  - 0: No confidence level set
  - 1: Low confidence
  - 2: Reserved
  - 3: High confidence
- Bits 14-15: Cirrus Confidence
  - 0: No confidence level set
  - 1: Low confidence
  - 2: Reserved
  - 3: High confidence

In [48]:
image=ls8.filterBounds(aoi).filterDate('2023-01-01','2023-02-01').first()
qa_pixel=image.select('QA_PIXEL')

qa_pixel.getInfo()

{'type': 'Image',
 'bands': [{'id': 'QA_PIXEL',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7531, 7681],
   'crs': 'EPSG:32650',
   'crs_transform': [30, 0, 675285, 0, -30, 3471315]}],
 'id': 'LANDSAT/LC08/C02/T1_RT/LC08_119039_20230104',
 'version': 1673514163115520.0,
 'properties': {'RADIANCE_MULT_BAND_5': 0.0063233,
  'RADIANCE_MULT_BAND_6': 0.0015726,
  'RADIANCE_MULT_BAND_3': 0.012254,
  'RADIANCE_MULT_BAND_4': 0.010333,
  'RADIANCE_MULT_BAND_1': 0.012986,
  'RADIANCE_MULT_BAND_2': 0.013298,
  'WRS_TYPE': 2,
  'K2_CONSTANT_BAND_11': 1201.1442,
  'K2_CONSTANT_BAND_10': 1321.0789,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[120.7468753601711, 29.32123525722844],
    [120.75436074251996, 29.348612033328703],
    [120.7860261076601, 29.465769723260998],
    [120.83261078888772, 29.638178788879483],
    [120.85887716110015, 29.735465316757885],
    [120.92299500807505, 29.972495675207192],
    [1

可视化一下看看

In [49]:
from IPython.display import Image

url=qa_pixel.getThumbUrl({
    'min':0,
    'max':1,
    'palette':'000000,FF0000,00FF00,0000FF',
    'region':aoi,
    'dimensions':'512'
})

Image(url=url)

有点奇怪,通过ipyleaflet看看到底是怎么回事

In [50]:
def display(image,map,  name, centered=False):
    bns = image.bandNames().length().getInfo()
    if bns == 3:
        image = image.rename('B1', 'B2', 'B3')
        pb_99 = ['B1_p99', 'B2_p99', 'B3_p99']
        pb_1 = ['B1_p1', 'B2_p1', 'B3_p1']
        img = ee.Image.rgb(image.select('B1'), image.select('B2'), image.select('B3'))
    else:
        image = image.rename('B1')
        pb_99 = ['B1_p99']
        pb_1 = ['B1_p1']
        img = image.select('B1')
    percentiles = image.reduceRegion(ee.Reducer.percentile([1, 99]), maxPixels=1e11)
    mx = percentiles.values(pb_99)
    if centered:
        mn = ee.Array(mx).multiply(-1).toList()
    else:
        mn = percentiles.values(pb_1)
    map.addLayer(img, {'min': mn, 'max': mx}, name)

map1=Map(center=[35, 110], zoom=4,basemap=ipyl.basemaps.Esri.WorldImagery)
display(qa_pixel, map1, 'image')
map1


Map(center=[35, 110], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

之前计算aoi的中心点出现了一些报错,下面是一些实验性质的代码

In [29]:
polygon = aoi

# 计算多边形的质心
centroid = polygon.centroid()

# 获取质心的坐标
coordinates = centroid.coordinates()

# 打印坐标
print('质心坐标:', coordinates.getInfo())

质心坐标: [104.42466128146167, 34.80447663804673]


真的是越看越奇怪啊

让我把数据导出来试试

In [57]:
data = qa_pixel.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=aoi,
    scale=100,
    maxPixels=1e11
)

# 创建一个导出任务
task = ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection([ee.Feature(None, {'data': data.get('QA_PIXEL')})]),
    description='exportBandData',
    fileFormat='CSV'
)

# 开始执行导出任务
task.start()


通过time模块监视task进程

In [58]:
import time

while task.active():
    status = task.status()
    print(status)

    # 等待一段时间后再次检查状态
    time.sleep(5)

task.status()


{'state': 'READY', 'description': 'exportBandData', 'creation_timestamp_ms': 1703175573632, 'update_timestamp_ms': 1703175573632, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'G37G5U6H4PWDPYJKXRJD67ES', 'name': 'projects/earthengine-legacy/operations/G37G5U6H4PWDPYJKXRJD67ES'}


{'state': 'COMPLETED',
 'description': 'exportBandData',
 'creation_timestamp_ms': 1703175573632,
 'update_timestamp_ms': 1703175578575,
 'start_timestamp_ms': 1703175577037,
 'task_type': 'EXPORT_FEATURES',
 'destination_uris': ['https://drive.google.com/'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 0.11607155203819275,
 'id': 'G37G5U6H4PWDPYJKXRJD67ES',
 'name': 'projects/earthengine-legacy/operations/G37G5U6H4PWDPYJKXRJD67ES'}

得到的结果真的很奇怪, 几乎空白的表格...

有点绝望

问问GPT, 换个路子试试

In [68]:
# 定义时间范围和位置
startDate = '2023-01-01'
endDate = '2023-02-01'
point = centroid

# 加载 Landsat 8 图像集
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA") \
    .filterDate(startDate, endDate) \
    .filterBounds(point)

# 选择一个图像
image = l8.first()  # 或者使用其他方法选择特定图像

# 定义一个函数来计算每个图像的 CloudScore
def addCloudScore(image):
    cloudScore = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')
    return image.addBands(cloudScore)

# 将云掩膜算法应用于图像集合
withCloudScores = l8.map(addCloudScore)

# 计算平均 CloudScore
meanCloudScore = withCloudScores.select('cloud').mean()

# 提取指定位置的平均 CloudScore
averageCloudInfo = meanCloudScore.reduceRegion(
    reducer=ee.Reducer.first(),
    geometry=point,
    scale=30
)

# 输出平均 CloudScore
print(averageCloudInfo.getInfo())

# 计算图像集合中的图像数量
imageCount = l8.size()

# 输出图像数量
print('该点在指定月份内的 Landsat 8 图像数量:', imageCount.getInfo())

{'cloud': 56}
该点在指定月份内的 Landsat 8 图像数量: 2


先浅浅放下Landsat8,让我们看看从GPT4中问到的更好的产品之一:MODIS AQUA的产品

ee.ImageCollection("MODIS/061/MYD08_M3")